# Denoising single-cell RNA-seq datasets using deep autoencoders

The purpose of this project is to create a method that uses gene correlations to uncover biologically relevant genes within a single cell RNA-seq dataset. 

The pipeline:
1. Feed data into neural network (autoencoder) to get low dimensional representation (done locally in python). This step inputs an M-by-N matrix and outputs a k-by-N matrix (lower dimensional representation).
2. Create a similarity matrix between genes using the clean representations (done on server in python). This step inputs a k-by-N matrix and outputs a N-by-N matrix. 
3. Robust PCA with a Laplacian regularization term (done on server in matlab). This step inputs a M-by-N matrix (original matrix) and outputs a M-by-N low-rank approximation.
4. Cluster on the low-rank matrix (python on server). 

In [1]:
# Use scikit-learn to perform clustering

from sklearn import cluster,manifold
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.cm as cm
    
# obtain labels via spectral clustering
def jz_spectral(X,k):
    spectral = cluster.SpectralClustering(n_clusters=k)
    spectral.fit(X)
    labels = spectral.labels_
    return labels

# obtain labels via spectral clustering after SVD
def jz_spectral_svd(X,k):
    u,s,v = np.linalg.svd(X, full_matrices=0, compute_uv=1)
    return jz_spectral(u[:,0:(k+1)],k)

# obtain labels via kmeans
def jz_kmeans(X,k,num_iter=50):
    k_means = cluster.KMeans(n_clusters=k,max_iter=num_iter,precompute_distances=True)
    k_means.fit(X)
    labels = k_means.labels_
    return labels

# Plot function with colors corresponding to labels (first two columns of X)
def jz_plot(X,labels,title):
    unique_labels = np.unique(labels)
#     plt.figure(figsize=(15,10))
    x = np.arange(len(unique_labels))
    ys = [i+x+(i*x)**2 for i in range(len(unique_labels))]
    colors = cm.rainbow(np.linspace(0, 1, len(ys)))
    i = 0
    for label in unique_labels:
        ind = np.squeeze(labels == label)
        plt.scatter(X[ind,0],X[ind,1],c=colors[i])
        i += 1
    plt.title(title)
    plt.show()
    
# obtain two major directions from t-SNE
def jz_tSNE(X):
    tsne = manifold.TSNE(n_components=2);
    X_tsne = tsne.fit_transform(X);
    return X_tsne

# obtain two major directions from t-SNE
def jz_PCA(X):
    pca = PCA(n_components=2);
    X_pca = pca.fit_transform(X);
    return X_pca

In [4]:
# Load data
import numpy as np
# lab = 'Zeisel'
lab = 'Buettner'
X = np.loadtxt('./data/'+lab+'/'+lab+'_expression.txt')
M = np.shape(X)[0]
N = np.shape(X)[1]
print M,N

182 8989


In [6]:
# Simply dimensionality reduction PCA
X_PCA = jz_PCA(X)
np.savetxt('./Xk_Buettner_PCA.txt',X_PCA.T,delimiter='\t')

In [15]:
%%time
# Train a two-layer autoencoder using Theanets. Train all layers simultaneously.
# model.train input should be M-by-N, where M is the number of training example

import theanets

# Train first layer
n1 = M
n2 = int(np.round(n1**.65))
n3 = int(np.round(n2**.65))
model = theanets.Autoencoder([n1,n2,n3,n2,n1])
dropout = 0.2
model.train([X.T],input_dropout=dropout)

# Train second layer
Xk = model.encode(X.T)
print np.shape(Xk)

(8989, 9)
CPU times: user 5min 5s, sys: 1min, total: 6min 6s
Wall time: 5min 48s


In [11]:
%%time
# Train a two-layer autoencoder using Theanets. Train each layer separately
# model.train input should be M-by-N, where M is the number of training example

import theanets

# Train first layer
n1 = M
n2 = int(np.round(n1**.65))
model = theanets.Autoencoder([n1,n2,n1])
dropout = 0.2
model.train([X.T],input_dropout=dropout)

# Encode data after training first layer
X1 = model.encode(X.T)
print np.shape(X1)

# Train second layer
n3 = int(np.round(n2**.65))
model2 = theanets.Autoencoder([n2,n3,n2])
dropout = 0.2
model2.train([X1],input_dropout=dropout)

# Full encoding
Xk = model2.encode(X1)
print np.shape(Xk)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11 µs


In [25]:
# Train a 3-layer autoencoder using Keras.

from keras.models import Sequential
from keras.layers import containers
from keras.layers.core import Dense, Dropout, Activation, AutoEncoder
from keras.optimizers import SGD,RMSprop

def keras_3layer_autoencoder(X_train,dropout,n1,n2,n3,batch_size,nb_epoch,activation,out_rec=True):
    # input shape: (nb_samples, M)
    encoder = containers.Sequential([Dense(n2, input_dim=n1, activation=activation), 
                                     Dense(n3, activation=activation)])
    decoder = containers.Sequential([Dense(n2, input_dim=n3, activation=activation), 
                                     Dense(n1, activation=activation)])

    autoencoder = Sequential()
    autoencoder.add(AutoEncoder(encoder=encoder, decoder=decoder, output_reconstruction=out_rec))
    autoencoder.add(Dropout(dropout))

    autoencoder.compile(loss='mean_squared_error', optimizer='rmsprop')
    autoencoder.fit(X_train, X_train, verbose=2, batch_size=batch_size, nb_epoch=nb_epoch, shuffle=True) 

    return autoencoder

def keras_4layer_autoencoder(X_train,dropout,n1,n2,n3,n4,batch_size,nb_epoch,activation,out_rec=True):
    # input shape: (nb_samples, M)
    encoder = containers.Sequential([Dense(n2, input_dim=n1, activation=activation), 
                                     Dense(n3, activation=activation), 
                                     Dense(n4, activation = activation)])
    decoder = containers.Sequential([Dense(n3, input_dim=n4, activation=activation),
                                     Dense(n2, activation=activation), 
                                     Dense(n1, activation=activation)])

    autoencoder = Sequential()
    autoencoder.add(AutoEncoder(encoder=encoder, decoder=decoder, output_reconstruction=out_rec))
    autoencoder.add(Dropout(dropout))

    autoencoder.compile(loss='mean_squared_error', optimizer='rmsprop')
    autoencoder.fit(X_train, X_train, verbose=0, batch_size=batch_size, nb_epoch=nb_epoch, shuffle=True) 

    return autoencoder

def keras_5layer_autoencoder(X_train,dropout,n1,n2,n3,n4,n5,batch_size,nb_epoch,activation,out_rec=True):
    # input shape: (nb_samples, M)
    encoder = containers.Sequential([Dense(n2, input_dim=n1, activation=activation), 
                                     Dense(n3, activation=activation), 
                                     Dense(n4, activation=activation),
                                     Dense(n5, activation=activation)])
    decoder = containers.Sequential([Dense(n4, input_dim=n5, activation=activation),
                                     Dense(n3, activation=activation),
                                     Dense(n2, activation=activation), 
                                     Dense(n1, activation=activation)])

    autoencoder = Sequential()
    autoencoder.add(AutoEncoder(encoder=encoder, decoder=decoder, output_reconstruction=out_rec))
    autoencoder.add(Dropout(dropout))

    autoencoder.compile(loss='mean_squared_error', optimizer='rmsprop')
    autoencoder.fit(X_train, X_train, verbose=0, batch_size=batch_size, nb_epoch=nb_epoch, shuffle=True) 

    return autoencoder

In [106]:
# Test multiple hyperparameters for 5 layer neural net (Keras)

import time

dropout = 0.2
n1 = M
batch_size = M
nb_epoch = 200
X_train = X.T

n2 = 120
n3 = 40
n4 = 30
n5 = 5
activation = 'tanh'

start_time = time.time()
for i in range(40):
    autoencoder = keras_5layer_autoencoder(X_train,dropout,n1,n2,n3,n4,n5,batch_size,nb_epoch,activation,out_rec=True)

    # Save encoded then decoded result
    Xk = autoencoder.predict(X_train)
    print np.shape(Xk)
    flname = './Buettner_5layer_keras_rep/X_Buettner_keras_'+activation+'_'+str(n2)+'hidden1_'+str(n3)+'hidden2_'+str(n4)+'hidden3_'+str(n5)+'_'+str(i)+'_recon.txt'
    np.savetxt(flname,Xk.T,delimiter='\t')

    # Save encoded result
    encoder = Sequential()
    encoder.add(Dense(n2,input_dim = n1, weights = autoencoder.layers[0].get_weights()[0:2], activation = activation))#, activation = activation))
    encoder.add(Dense(n3, weights = autoencoder.layers[0].get_weights()[2:4], activation = activation))
    encoder.add(Dense(n4, weights = autoencoder.layers[0].get_weights()[4:6], activation = activation))
    encoder.add(Dense(n5, weights = autoencoder.layers[0].get_weights()[6:8], activation = activation))
    encoder.compile(loss='mean_squared_error', optimizer='rmsprop')

    Xe = encoder.predict(X_train)
    flname = './Buettner_5layer_keras_rep/Xk_Buettner_keras_'+activation+'_'+str(n2)+'hidden1_'+str(n3)+'hidden2_'+str(n4)+'hidden3_'+str(n5)+'_'+str(i)+'_recon.txt'
    np.savetxt(flname,Xe.T,delimiter='\t')

    # Decoder for sanity check
    decoder = Sequential()
    decoder.add(Dense(n4,input_dim = n5, weights = autoencoder.layers[0].get_weights()[8:10], activation = activation))#, activation = activation))
    decoder.add(Dense(n3, weights = autoencoder.layers[0].get_weights()[10:12], activation = activation))
    decoder.add(Dense(n2, weights = autoencoder.layers[0].get_weights()[12:14], activation = activation))
    decoder.add(Dense(n1, weights = autoencoder.layers[0].get_weights()[14:16], activation = activation))
    decoder.compile(loss='mean_squared_error', optimizer='rmsprop')
    
    print np.sum(np.abs(decoder.predict(Xe)-Xk))
    print str(i) + ': ' + str(time.time()-start_time)
    print '-'*80
                    
import os

os.system('scp -r Buettner_5layer_keras_rep/ jessez@shannon.stanford.edu:/home/jessez/Classes/CS221_final_project')

(8989, 182)
0.0
0: 118.767498016
--------------------------------------------------------------------------------
(8989, 182)
0.0
1: 239.936073065
--------------------------------------------------------------------------------
(8989, 182)
0.0
2: 374.160480022
--------------------------------------------------------------------------------
(8989, 182)
0.0
3: 505.591223001
--------------------------------------------------------------------------------
(8989, 182)
0.0
4: 614.778312922
--------------------------------------------------------------------------------
(8989, 182)
0.0
5: 723.959218979
--------------------------------------------------------------------------------
(8989, 182)
0.0
6: 832.020348072
--------------------------------------------------------------------------------
(8989, 182)
0.0
7: 943.900252104
--------------------------------------------------------------------------------
(8989, 182)
0.0
8: 1053.27193499
-------------------------------------------------------

0

In [12]:
# Test multiple hyperparameters for 4 layer neural net (Keras)

import time

dropout = 0.2
n1 = M
batch_size = M
nb_epoch = 200
X_train = X.T

start_time = time.time()
for activation in ['linear','tanh','relu']:
    for n2 in [50,80,120]:
        for n3 in [10,20,40]:
            for n4 in [2,3,5]:

                autoencoder = keras_4layer_autoencoder(X_train,dropout,n1,n2,n3,n4,batch_size,nb_epoch,activation)

                Xk = autoencoder.predict(X_train)
                print np.shape(Xk)

                # Save encoded result
                flname = './Buettner_4layer_keras/X_Buettner_keras_'+activation+'_'+str(n2)+'hidden1_'+str(n3)+'hidden2_'+str(n4)+'recon.txt'
                np.savetxt(flname,Xk.T,delimiter='\t')

                print time.time()-start_time
                print '-'*80
                

import os

os.system('scp -r Buettner_4layer_keras/ jessez@shannon.stanford.edu:/home/jessez/Classes/CS221_final_project')

Epoch 1/200
0s - loss: 1.4246
Epoch 2/200
0s - loss: 1.3135
Epoch 3/200
0s - loss: 1.2883
Epoch 4/200
0s - loss: 1.2804
Epoch 5/200
0s - loss: 1.2653
Epoch 6/200
0s - loss: 1.2648
Epoch 7/200
0s - loss: 1.2620
Epoch 8/200
0s - loss: 1.2541
Epoch 9/200
0s - loss: 1.2537
Epoch 10/200
0s - loss: 1.2519
Epoch 11/200
0s - loss: 1.2516
Epoch 12/200
0s - loss: 1.2388
Epoch 13/200
0s - loss: 1.2209
Epoch 14/200
0s - loss: 1.2142
Epoch 15/200
0s - loss: 1.2023
Epoch 16/200
0s - loss: 1.1955
Epoch 17/200
0s - loss: 1.1872
Epoch 18/200
0s - loss: 1.1839
Epoch 19/200
0s - loss: 1.1815
Epoch 20/200
0s - loss: 1.1650
Epoch 21/200
0s - loss: 1.1470
Epoch 22/200
0s - loss: 1.1417
Epoch 23/200
0s - loss: 1.1352
Epoch 24/200
0s - loss: 1.1278
Epoch 25/200
0s - loss: 1.0982
Epoch 26/200
0s - loss: 1.0808
Epoch 27/200
0s - loss: 1.0805
Epoch 28/200
0s - loss: 1.0803
Epoch 29/200
0s - loss: 1.0787
Epoch 30/200
0s - loss: 1.0778
Epoch 31/200
0s - loss: 1.0678
Epoch 32/200
0s - loss: 1.0663
Epoch 33/200
0s -

In [ ]:
# Test multiple hyperparameters for 3 layer neural net (Keras)

dropout = 0.2
n1 = M
batch_size = M
nb_epoch = 200
X_train = X.T

for activation in ['linear','tanh','relu','sigmoid']:
    for n2 in [50,90]:
        for n3 in [3,5,10,20]:

            autoencoder = keras_3layer_autoencoder(X_train,dropout,n1,n2,n3,batch_size,nb_epoch,activation)

            Xk = autoencoder.predict(X_train)
            print np.shape(Xk)

            # Save encoded result
            flname = './test_3layer_keras/X_Buettner_keras_'+activation+'_'+str(n2)+'hidden1_'+str(n3)+'hidden2_recon.txt'
            np.savetxt(flname,Xk.T,delimiter='\t')

            print time.time()-start_time
            print '-'*80

Epoch 1/200
8989/8989 [==============================] - 0s - loss: 1.5669     
Epoch 2/200
8989/8989 [==============================] - 0s - loss: 1.0073     
Epoch 3/200
8989/8989 [==============================] - 0s - loss: 0.9852     
Epoch 4/200
8989/8989 [==============================] - 0s - loss: 0.9832     
Epoch 5/200
8989/8989 [==============================] - 0s - loss: 0.9785     
Epoch 6/200
8989/8989 [==============================] - 0s - loss: 0.9733     
Epoch 7/200
8989/8989 [==============================] - 0s - loss: 0.9706     
Epoch 8/200
8989/8989 [==============================] - 0s - loss: 0.9730     
Epoch 9/200
8989/8989 [==============================] - 0s - loss: 0.9670     
Epoch 10/200
8989/8989 [==============================] - 0s - loss: 0.9673     
Epoch 11/200
8989/8989 [==============================] - 0s - loss: 0.9653     
Epoch 12/200
8989/8989 [==============================] - 0s - loss: 0.9625     
Epoch 13/200
8989/8989 [=============

In [ ]:
# Test multiple hyperparameters for 2 layer neural net (Keras)

from keras.models import Sequential
from keras.layers import containers
from keras.layers.core import Dense, Dropout, Activation, AutoEncoder
from keras.optimizers import SGD,RMSprop
import os
import time

n1 = M
batch_size = M
nb_epoch = 50
X_train = X.T
start_time = time.time()
for activation in ['linear','tanh','relu','sigmoid']:
    for n2 in [3,5,7,10,20,50,90]:
        for dropout in [0.2]:
            
            # input shape: (nb_samples, M)
            encoder = Dense(n2, input_dim=n1, activation=activation)
            decoder = Dense(n1, input_dim=n2, activation=activation)

            autoencoder = Sequential()
            autoencoder.add(AutoEncoder(encoder=encoder, decoder=decoder, output_reconstruction=True))
            autoencoder.add(Dropout(dropout))

            autoencoder.compile(loss='mean_squared_error', optimizer='rmsprop')
            autoencoder.fit(X_train, X_train, verbose=2, batch_size=batch_size, nb_epoch=nb_epoch, shuffle=True) 

            Xk = autoencoder.predict(X_train)

            # Save encoded result
            flname = './test_2layer_keras/X_Buettner_keras_'+activation+'_'+str(n2)+'hidden_'+str(dropout)+'dropout_recon.txt'
            np.savetxt(flname,Xk.T,delimiter='\t')

            print time.time()-start_time
            print '-'*80

    # Move this file to server
#     os.system('scp '+flname+' jessez@shannon.stanford.edu:/home/jessez/Classes/CS221_final_project')

Epoch 1/50
8989/8989 [==============================] - 0s - loss: 2.2911     
Epoch 2/50
8989/8989 [==============================] - 0s - loss: 1.4964     
Epoch 3/50
8989/8989 [==============================] - 0s - loss: 1.0904     
Epoch 4/50
8989/8989 [==============================] - 0s - loss: 0.9880     
Epoch 5/50
8989/8989 [==============================] - 0s - loss: 0.9744     
Epoch 6/50
8989/8989 [==============================] - 0s - loss: 0.9720     
Epoch 7/50
8989/8989 [==============================] - 0s - loss: 0.9662     
Epoch 8/50
8989/8989 [==============================] - 0s - loss: 0.9645     
Epoch 9/50
8989/8989 [==============================] - 0s - loss: 0.9603     
Epoch 10/50
8989/8989 [==============================] - 0s - loss: 0.9577     
Epoch 11/50
8989/8989 [==============================] - 0s - loss: 0.9614     
Epoch 12/50
8989/8989 [==============================] - 0s - loss: 0.9600     
Epoch 13/50
8989/8989 [==========================